In [2]:
# Exercise 3: GridSearchCV

In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [ ]:
# Load data
housing = fetch_california_housing()
X, y = housing['data'], housing['target']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, shuffle=True, random_state=43
)

# Define model
rf = RandomForestRegressor(random_state=43)

# Parameter grid (minimum 3 values each)
param_grid = {
    'max_depth': [5, 10, 20],          # You can expand later
    'n_estimators': [10, 50, 100]
}

# GridSearchCV with 5-fold CV, MSE scoring
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error',  # sklearn minimizes negative MSE
    n_jobs=-1,  # Use all CPUs
    verbose=1
)

# Fit
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
